In [8]:
# %pip install haystack-ai
%pip install google-vertex-haystack
%pip install git+https://github.com/deepset-ai/haystack.git@main

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
from haystack_integrations.components.generators.google_vertex import VertexAIGeminiGenerator
# import Part of Google Vertex AI
from vertexai.generative_models import Part

## Defining a Schema to Parse the JSON Object

In [2]:
from typing import List
from pydantic import BaseModel, validator
from datetime import datetime

class audioDescription(BaseModel):
    time: datetime
    content: str
    @validator('time', pre=True)
    def parse_time(cls, value):
        return datetime.strptime(value, "%H:%M:%S.%f")

class audioDescriptionList(BaseModel):
    audioDescriptionList: List[audioDescription]

In [3]:
json_schema = audioDescriptionList.schema_json(indent=2)

## Creating a Custom Component: OutputValidator

In [4]:
import json
import random
import pydantic
from pydantic import ValidationError
from typing import Optional, List
from colorama import Fore
from haystack import component
import re

# Define the component input parameters
@component
class OutputValidator:
    def __init__(self, pydantic_model: pydantic.BaseModel):
        self.pydantic_model = pydantic_model
        self.iteration_counter = 0

    # Define the component output
    @component.output_types(valid_replies=List[str], invalid_replies=Optional[List[str]], error_message=Optional[str])
    def run(self, replies: List[str]):
        self.iteration_counter += 1

        # Try to parse the LLM's reply
        try:
            json_match = re.search(r'```json\s*([\s\S]*?)\s*```', replies[0])
            if json_match is None:
                json_match = re.search(r'```python\s*([\s\S]*?)\s*```', replies[0])
                if json_match is None:
                    raise ValueError("No JSON block found in the LLM's reply")
            
            output_dict = json.loads(json_match.group(1))
            replies[0] = json_match.group(1)
            self.pydantic_model.parse_obj(output_dict)
            
            print(
                Fore.GREEN
                + f"OutputValidator at Iteration {self.iteration_counter}: Valid JSON from LLM - No need for looping: {replies[0]}"
            )
            return {"valid_replies": replies}

        # Handle invalid JSON or other errors
        except (ValueError, ValidationError) as e:
            print(
                Fore.RED
                + f"OutputValidator at Iteration {self.iteration_counter}: Invalid JSON from LLM - Let's try again.\n"
                f"Output from LLM:\n {replies[0]} \n"
                f"Error from OutputValidator: {e}"
            )
            return {"invalid_replies": replies, "error_message": str(e)}

In [5]:
output_validator = OutputValidator(pydantic_model=audioDescriptionList)

## Creating the Prompt

In [6]:
from haystack.components.builders import PromptBuilder

prompt_template = """
參考以下影片
僅依照提供的資料，創建一個JSON文件，其中包含一個audioDescriptions列表，每個audioDescriptions對象包含一個時間戳(hh:mm:ss.ms)和繁體中文口述影像。時間戳應該是一個日期時間對象，內容應該是一個字符串。例如：
{{schema}}
僅使用提供的資料，不要添加任何其他資訊並確保您的答案符合格式要求，確保回覆是dict類型。
{% if invalid_replies and error_message %}
您已經在先前的嘗試中建立了以下輸出：{{invalid_replies}}
但是，這不符合上面的格式要求並觸發了此 Python 異常：{{error_message}}
更正輸出並重試。只需返回正確的輸出，無需任何額外的解釋。
{% endif %}
"""

prompt_builder = PromptBuilder(template=prompt_template)

In [7]:
# 在prompt中加入影片
@component
class AddVideo2Prompt:
    # [
    #     Part.from_uri(
    #         "gs://gemini-ad-gen/AD001.mp4", mime_type="video/mp4"
    #     ),
    #     prompt
    # ]
    def __init__(self, prompt: str):
        self.prompt = prompt

    @component.output_types(prompt=list)
    def run(self,schema: str , video_uri: str, invalid_replies: Optional[List[str]] = None, error_message: Optional[str] = None):
        prompt = prompt_builder.run(schema=schema, invalid_replies=invalid_replies, error_message=error_message)
        return {"prompt": [Part.from_uri(video_uri, mime_type="video/mp4"),prompt["prompt"]]}


In [8]:
add_video_2_prompt = AddVideo2Prompt(prompt=prompt_builder)

In [9]:
@component
class GeminiGenerator:
    def __init__(self, project_id, location, model):
        self.project_id = project_id
        self.location = location
        self.model = model
    
    @component.output_types(replies=List[str])
    def run(self, prompt: List):
        generator = VertexAIGeminiGenerator(project_id=self.project_id, location=self.location, model=self.model)
        return {"replies": generator.run(prompt)["replies"]}

In [10]:
gemini_generator = GeminiGenerator(project_id="gemini-rain-py", location="us-central1", model="gemini-1.5-pro-preview-0514")

In [11]:
from haystack import Pipeline
pipeline = Pipeline(max_loops_allowed=5)

# Add components to your pipeline
# pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=add_video_2_prompt, name="add_video")
pipeline.add_component(instance=gemini_generator, name="llm")
pipeline.add_component(instance=output_validator, name="output_validator")

# Now, connect the components to each other
# pipeline.connect("prompt_builder", "add_video")
pipeline.connect("add_video.prompt", "llm")
# pipeline.connect("prompt_builder", "llm")
pipeline.connect("llm", "output_validator")
# # If a component has more than one output or input, explicitly specify the connections:
pipeline.connect("output_validator.invalid_replies", "add_video.invalid_replies")
pipeline.connect("output_validator.error_message", "add_video.error_message")

🚅 Components
  - add_video: AddVideo2Prompt
  - llm: GeminiGenerator
  - output_validator: OutputValidator
🛤️ Connections
  - add_video.prompt -> llm.prompt (list)
  - llm.replies -> output_validator.replies (List[str])
  - output_validator.invalid_replies -> add_video.invalid_replies (Optional[List[str]])
  - output_validator.error_message -> add_video.error_message (Optional[str])

In [12]:
pipeline.draw("auto-correct-pipeline.png")

In [13]:

uri = "gs://gemini-ad-gen/AD001.mp4"
result = pipeline.run({
    "add_video": {"schema": json_schema, "video_uri": uri},
})


OutputValidator at Iteration 1: Valid JSON from LLM - No need for looping: {
  "audioDescriptionList": [
    {
      "time": "00:00:00.000",
      "content": "一個女人走在地鐵站裡"
    },
    {
      "time": "00:00:03.000",
      "content": "一個老婦人焦急地在人群中尋找"
    },
    {
      "time": "00:00:04.000",
      "content": "電子螢幕顯示時間和醫院"
    },
    {
      "time": "00:00:05.000",
      "content": "老婦人向路人詢問是否看到小偉"
    },
    {
      "time": "00:00:07.000",
      "content": "地鐵站人來人往"
    },
    {
      "time": "00:00:08.000",
      "content": "地鐵進站，人們上下車"
    }
  ]
}


In [14]:
result

{'output_validator': {'valid_replies': ['```python\n{\n  "audioDescriptionList": [\n    {\n      "time": "00:00:00.000",\n      "content": "一位女士走過鋪著瓷磚的人行道。"\n    },\n    {\n      "time": "00:00:01.000",\n      "content": "地鐵呼嘯而過，人們在擁擠的月台上走動。"\n    },\n    {\n      "time": "00:00:03.000",\n      "content": "一位年長的女士，面帶擔憂的表情，在尋找什麼人。"\n    },\n    {\n      "time": "00:00:04.000",\n      "content": "數位顯示幕顯示「07:20醫院」。螢幕下方顯示繁體中文「請問有沒有人看到我家韋韋…」和英文「Please, have you seen little Wade?」。"\n    },\n    {\n      "time": "00:00:05.000",\n      "content": "年長的女士焦急地詢問周圍的人，一位年輕女子走過。"\n    },\n    {\n      "time": "00:00:07.000",\n      "content": "從月台上方的廣角鏡頭可以看到擁擠的人群，地鐵進站了。"\n    },\n    {\n      "time": "00:00:09.000",\n      "content": "人們排隊等候上地鐵，年長的女士仍然焦急地在人群中尋找。"\n    }\n  ]\n}\n```']}}

In [15]:
valid_reply = result["output_validator"]["valid_replies"][0]
valid_json = json.loads(valid_reply)
print(valid_json)

{'audioDescriptionList': [{'time': '00:00:00.000', 'content': '一個女人走在地鐵站裡'}, {'time': '00:00:03.000', 'content': '一個老婦人焦急地在人群中尋找'}, {'time': '00:00:04.000', 'content': '電子螢幕顯示時間和醫院'}, {'time': '00:00:05.000', 'content': '老婦人向路人詢問是否看到小偉'}, {'time': '00:00:07.000', 'content': '地鐵站人來人往'}, {'time': '00:00:08.000', 'content': '地鐵進站，人們上下車'}]}
